### Objective

In this notebook, we generate samples for the input paramters for thermal analysis. Here, we condsider two heat sources.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import qmc
from sklearn.cluster import KMeans
from collections import defaultdict

In [2]:
# Define parameters
Ta = 25
c_source = 50e-3
d_source = 65e-3
c_module = 61.4e-3
d_module = 106e-3
fan = 3

# Generate samples
samlple_num = 10000
dim = 8
append_position_number = 1
sampler = qmc.LatinHypercube(d=dim)
X = sampler.random(n=samlple_num)

In [3]:
# Scale X to the required ranges
Q1_min, Q1_max = 50, 400
Q1 = (Q1_max - Q1_min) * X[:, 0] + Q1_min

Q2_min, Q2_max = 50, 400
Q2 = (Q2_max - Q2_min) * X[:, 1] + Q2_min

d_min, d_max = 5e-3, 30e-3
d = (d_max - d_min) * X[:, 2] + d_min

b_min, b_max = 73.7e-3, 307e-3
b = (b_max - b_min) * X[:, 3] + b_min

L_min, L_max = 127.2e-3, 530e-3
L = (L_max - L_min) * X[:, 4] + L_min

c_min, c_max = 10e-3, 39e-3
c = (c_max - c_min) * X[:, 5] + c_min

L_duct_min, L_duct_max = 20e-3, 50e-3
L_duct = (L_duct_max - L_duct_min) * X[:, 6] + L_duct_min

n_min, n_max = 10, 50
n = np.round((n_max - n_min) * X[:, 7] + n_min).astype(int)

samples = np.column_stack((Q1, Q2, d, b, L, c, L_duct, n))

In [4]:
result_design = []
t_invalid = 0
valid_position = defaultdict(list)

# Derived parameters calculation
for i, sample in enumerate(samples):
    if (i+1)%500 == 0:
        print(f"Processing {i+1} samples")
        
    Q1, Q2, d, b, L, c, L_duct, n = sample

    t_min = 1e-3
    t_max = b / n - 1e-3

    if t_min > t_max:
        t_invalid += 1
    
    else:
        t = np.random.rand() * (t_max - t_min) + t_min

        Xc_min = c_module / 2
        Xc_max = b - c_module / 2
        Yc_min = d_module / 2
        Yc_max = L - d_module / 2

        # Generate position samples 
        position_sampler = qmc.LatinHypercube(d=4)
        positions = position_sampler.random(n=5000)

        # Scale samples
        positions[:, 0] = positions[:, 0]*(Xc_max-Xc_min)+ Xc_min;
        positions[:, 1] = positions[:, 1]*(Yc_max-Yc_min)+ Yc_min;
        positions[:, 2] = positions[:, 2]*(Xc_max-Xc_min)+ Xc_min;
        positions[:, 3] = positions[:, 3]*(Yc_max-Yc_min)+ Yc_min;

        # Check non-overlapping
        xc1, yc1, xc2, yc2 = positions[:, 0], positions[:, 1], positions[:, 2], positions[:, 3]
        non_overlapping = (np.abs(xc1 - xc2) > c_module) | (np.abs(yc1 - yc2) > d_module)

        # Retain valid positions
        valid_positions = positions[non_overlapping]
        valid_position['b'].append(b)
        valid_position['L'].append(L)
        valid_position['valid_pos'].append(len(valid_positions))

        # Compose the design variables
        if len(valid_positions) > 0:
            
            if append_position_number == 1:
                # Randomly pick 1 sample from the valid positions
                random_index = np.random.randint(0, len(valid_positions))
                selected_position = valid_positions[random_index]
                result_design.append(np.concatenate((sample, [t], selected_position)))
                
            elif len(valid_positions) <= append_position_number:
                for pos in valid_positions:
                    # Append each valid position to the design
                    result_design.append(np.concatenate((sample, [t], pos)))
    
            else:
                # Perform clustering and append the centroid/closest to centroid positions
                kmeans = KMeans(n_clusters=append_position_number, n_init=10, random_state=0).fit(valid_positions)
                centers = kmeans.cluster_centers_
                
                # For each center, find the closest valid position
                for center in centers:
                    distances = np.sqrt(((valid_positions - center)**2).sum(axis=1))
                    closest_index = np.argmin(distances)
                    closest_position = valid_positions[closest_index]
                    
                    # Append the closest valid position to the design
                    result_design.append(np.concatenate((sample, [t], closest_position)))

# Swap Q1 & Q2 values to ensure Q1 always larger than Q2
result_design = np.array(result_design)
swap = result_design[:, 0] < result_design[:, 1]
result_design[swap, 0], result_design[swap, 1] = result_design[swap, 1], result_design[swap, 0].copy()

# Swap locations as well
result_design[swap, -4], result_design[swap, -2] = result_design[swap, -2], result_design[swap, -4].copy()
result_design[swap, -3], result_design[swap, -1] = result_design[swap, -1], result_design[swap, -3].copy()

Processing 500 samples
Processing 1000 samples
Processing 1500 samples
Processing 2000 samples
Processing 2500 samples
Processing 3000 samples
Processing 3500 samples
Processing 4000 samples
Processing 4500 samples
Processing 5000 samples
Processing 5500 samples
Processing 6000 samples
Processing 6500 samples
Processing 7000 samples
Processing 7500 samples
Processing 8000 samples
Processing 8500 samples
Processing 9000 samples
Processing 9500 samples
Processing 10000 samples


In [5]:
df = pd.DataFrame(valid_position)

In [6]:
print(f"Invalid t design: {t_invalid}/{samlple_num}")
print(f"Invalid heat source position: {np.sum(df['valid_pos'] == 0)}/{len(df)}")
print(f"Invalid heat source position ratio (%): {np.sum(df['valid_pos'] == 0)/len(df)*100:.3f}%")
print(f"Remaining valid samples: {result_design.shape[0]}/{samlple_num}")

Invalid t design: 198/10000
Invalid heat source position: 414/9802
Invalid heat source position ratio (%): 4.224%
Remaining valid samples: 9388/10000


In [7]:
# Configure dataframe
design_df = pd.DataFrame(result_design)
column_names = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
design_df.columns = column_names
design_df

,Q1,Q2,d,b,L,c,L_duct,n,t,xc1,yc1,xc2,yc2
0,354.162049,209.973777,0.008376,0.105262,0.358018,0.024769,0.049020,40.0,0.001542,0.048407,0.056408,0.037934,0.304392
1,310.266548,304.103605,0.016987,0.252752,0.375389,0.025041,0.048750,22.0,0.005418,0.176011,0.056698,0.104327,0.227647
2,143.215680,90.987329,0.024670,0.087385,0.367372,0.011184,0.029552,35.0,0.001247,0.032683,0.289248,0.049411,0.181909
3,164.643507,157.346416,0.026123,0.167066,0.523121,0.024806,0.040541,18.0,0.005343,0.104320,0.229971,0.041316,0.270856
4,223.010215,162.737110,0.013772,0.212688,0.198952,0.022884,0.044573,31.0,0.005747,0.162109,0.140386,0.056991,0.064584
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9383,221.704797,140.655720,0.015763,0.177631,0.298198,0.036769,0.032717,33.0,0.003203,0.091877,0.237605,0.096537,0.054759
9384,348.057919,114.923554,0.011999,0.101835,0.298507,0.032715,0.026921,15.0,0.004623,0.046143,0.196469,0.042999,0.057228
9385,332.899604,220.776222,0.010925,0.265895,0.254384,0.013308,0.049156,33.0,0.003604,0.159820,0.198348,0.159719,0.053509
9386,384.975128,311.977766,0.017512,0.142966,0.168872,0.027084,0.042955,18.0,0.004502,0.043461,0.095763,0.109446,0.059088


In [9]:
# Save the design samples
design_df.to_csv("./dataset/AL_train_10000.csv", index=False)